In [1]:
import torch
import torch.nn as nn
import numpy as np

# 1.Przygotowanie danych 
def generate_data(num_samples=10000):
    seq_len = 24
    # Generowanie losowych liczb całkowitych
    A_int = np.random.randint(0, 2**seq_len, num_samples)
    B_int = np.random.randint(0, 2**seq_len, num_samples)
    
    # Obliczanie różnicy 
    diff_int = (A_int - B_int) % (2**seq_len)
    
    # Konwersja na postać binarną 
    def to_bit_list(n_array, length):
        return [list(map(int, np.binary_repr(n, length)[::-1])) for n in n_array]

    A_bits = to_bit_list(A_int, seq_len)
    B_bits = to_bit_list(B_int, seq_len)
    diff_bits = to_bit_list(diff_int, seq_len)
    
    #Formatowanie dla RNN: (Sample, Seq_Len, Features)
    X = np.stack([A_bits, B_bits], axis=2).astype(np.float32)
    Y = np.expand_dims(diff_bits, axis=2).astype(np.float32)
    
    return torch.from_numpy(X), torch.from_numpy(Y)

#2.Architektura Sieci RNN
class BinarySubtractionRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BinarySubtractionRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True, nonlinearity='tanh')
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid() # Wynik to prawdopodobieństwo bitu 1

    def forward(self, x):
        out, h_n = self.rnn(x)
        out = self.fc(out)
        out = self.sigmoid(out)
        return out

#3.Parametry i Trening
input_size = 2
hidden_size = 8  
output_size = 1
seq_len = 24

model = BinarySubtractionRNN(input_size, hidden_size, output_size)
criterion = nn.BCELoss() # Binary Cross Entropy
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

#Generowanie danych
X_train, Y_train = generate_data(8000)

#Pętla treningowa
for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    
    outputs = model(X_train)
    loss = criterion(outputs, Y_train)
    
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {loss.item():.4f}')

#4. Testowanie modelu
model.eval()
with torch.no_grad():
    X_test, Y_test = generate_data(1)
    prediction = model(X_test)
    predicted_bits = (prediction > 0.5).int()
    
    print("\n--- Test modelu ---")
    print(f"Liczba A (bits): {X_test[0,:,0].int().tolist()}")
    print(f"Liczba B (bits): {X_test[0,:,1].int().tolist()}")
    print(f"Prawdziwy wynik: {Y_test[0,:,0].int().tolist()}")
    print(f"Przewidywany:    {predicted_bits[0,:,0].tolist()}")

Epoch [10/100], Loss: 0.6934
Epoch [20/100], Loss: 0.6932
Epoch [30/100], Loss: 0.6930
Epoch [40/100], Loss: 0.6927
Epoch [50/100], Loss: 0.6921
Epoch [60/100], Loss: 0.6891
Epoch [70/100], Loss: 0.6789
Epoch [80/100], Loss: 0.6566
Epoch [90/100], Loss: 0.6140
Epoch [100/100], Loss: 0.5354

--- Test modelu ---
Liczba A (bits): [1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0]
Liczba B (bits): [0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0]
Prawdziwy wynik: [1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1]
Przewidywany:    [1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1]
